### Obejective : To determine what items are purchased together, sequentially or seasonally at a supermarket so as to improve inventory management and cross-selling, improve customer satisfaction and regulate staff rostering efficiently.

### Method:  Market Basket Analysis, a technique which identifies the strength of association between pairs of products purchased together and identify patterns of co-occurrence. 

Market Basket Analysis creates If-Then scenario rules, for example, if item A is purchased then item B is likely to be purchased. The rules are probabilistic in nature or, in other words, they are derived from the frequencies of co-occurrence in the observations. Frequency is the proportion of baskets that contain the items of interest. The rules can be used in pricing strategies, product placement, and various types of cross-selling strategies.

In [2]:
# Load necessary packages
import numpy as np
import pandas as pd
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
# Read data from web
data = pd.read_excel ('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
data.shape
# 541909 observations for 8 variables

(541909, 8)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540454 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 37.2+ MB


### Data Prep

In [11]:
# Data Cleaning
# Remove extra spaces, duplicates, convert invoice number to a string value, remove credit transactions
data['Description'] = data['Description'].str.strip()
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')] 
#Tilde means negation, i,e in this case, InvoiceNo that DOESN'T contain 'C'
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [13]:
data.shape

(532621, 8)

### Now the data is ready to be used

In [14]:
data['Country'].value_counts()
#count the number of unique values by country

United Kingdom          487622
Germany                   9042
France                    8408
EIRE                      7894
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Unspecified                446
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     295
Hong Kong                  284
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon 

Based on country transaction counts, UK has the highest number of transactions

Now looking at just Australian transactions:

In [17]:
# separate transactions for Australia by subsetting the data ( total of 1185 Australian transactions)
transaction_basket = (data[data['Country'] =='Australia']
            .groupby(['InvoiceNo', 'Description'])['Quantity']
            .sum().unstack().reset_index().fillna(0)
            .set_index('InvoiceNo'))

Encode values for 0 (indicates no transaction) and 1 ( existence of transaction) before proceeding with Market Basket Analysis

In [32]:
# convert all positive values to 1 and everything else to 0
def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

transaction_basket_sets = transaction_basket.applymap(encode_units)

#remove 'postage' as an item
transaction_basket_sets.drop('POSTAGE',inplace=True, axis =1)

# View the transaction basket
transaction_basket.head()

Description,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,...,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP VINTAGE LEAF DESIGN,WRAP WEDDING DAY,YELLOW GIANT GARDEN THERMOMETER
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537676,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Market Basket Analysis Steps:

To find the most frequent items bought:
    Set minimum threshold at 0.7
    Generate rules using the lift metric ( lift tells us how likely item Y is to be purchased with item x)
    -> Life >1 inidicates items are likely to be bought together, thus association rule is good.

In [34]:
# Generating frequent itemsets
frequent_itemsets = apriori(transaction_basket_sets,
                           min_support=0.07, use_colnames=True)

#Generating rules
rules = association_rules(frequent_itemsets, metric="lift",
min_threshold=1)

#view top 100 rules
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(RED RETROSPOT CAKE STAND),(36 PENCILS TUBE RED RETROSPOT),0.070175,0.070175,0.070175,1.000000,14.250000,0.065251,inf
1,(36 PENCILS TUBE RED RETROSPOT),(RED RETROSPOT CAKE STAND),0.070175,0.070175,0.070175,1.000000,14.250000,0.065251,inf
2,(SET OF 3 CAKE TINS PANTRY DESIGN),(36 PENCILS TUBE RED RETROSPOT),0.157895,0.070175,0.070175,0.444444,6.333333,0.059095,1.673684
3,(36 PENCILS TUBE RED RETROSPOT),(SET OF 3 CAKE TINS PANTRY DESIGN),0.070175,0.157895,0.070175,1.000000,6.333333,0.059095,inf
4,(SET OF 6 SOLDIER SKITTLES),(4 TRADITIONAL SPINNING TOPS),0.122807,0.087719,0.070175,0.571429,6.514286,0.059403,2.128655
5,(4 TRADITIONAL SPINNING TOPS),(SET OF 6 SOLDIER SKITTLES),0.087719,0.122807,0.070175,0.800000,6.514286,0.059403,4.385965
6,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.105263,0.105263,0.105263,1.000000,9.500000,0.094183,inf
7,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.105263,0.105263,0.105263,1.000000,9.500000,0.094183,inf
8,(RED DINER WALL CLOCK),(ALARM CLOCK BAKELIKE GREEN),0.070175,0.105263,0.070175,1.000000,9.500000,0.062789,inf
9,(ALARM CLOCK BAKELIKE GREEN),(RED DINER WALL CLOCK),0.105263,0.070175,0.070175,0.666667,9.500000,0.062789,2.789474


In [35]:
transaction_basket_sets['36 PENCILS TUBE RED RETROSPOT'].sum()


4

In [36]:
transaction_basket_sets['RED RETROSPOT CAKE STAND'].sum()

4

the transaction basket for this antecedent- consequent rule shows that the 4 people who purchased the ’36 PENCILS TUBE RED RETROSPOT’ were the ones who also purchased the ‘RED RETROSPOT CAKE STAND’.

### Recommendation: Place these 2 items together to increase the number of sales

## Visualise the results

In [37]:
# view results for one antecedent lhs item againast consequents
import seaborn as sns
import matplotlib.pyplot as plt